# Create Training Sets for Chemical Interpolation Test
The goal of this test is to determine whether machine learning models are able to infer the interactions between elements that are not included in the training set. We will set up two different tests, 
1. Exclude a single quaternary. Withhold all entries that contain exclusively elements from the target 
2. Exclude a certain pair interaction. Withhold all entries that contain both elements in a certain pair

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from pymatgen import Composition
from itertools import product
import numpy as np
import pandas as pd
import os
import shutil

## Read in the OQMD dataset
We want only the lowest-energy entry at each composition

In [2]:
oqmd_data = pd.read_csv(os.path.join('..', 'oqmd_all.txt'), delim_whitespace=True)
print('Read %d entries'%len(oqmd_data))
oqmd_data.head()

Read 506114 entries


C:\Users\ward6\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability
0,Li1,-1.892,17.8351,None,0.0,0.015186,0.0151862666667
1,Mg1,-1.5396,22.9639,None,0.0,0.002912,0.0029123775
2,Kr1,0.011256,41.4146,None,7.367,0.015315,0.015314775
3,Na1,-1.2991,32.9826,None,0.0,0.003780,0.00377956333333
4,Pd1,-5.15853,15.2088,None,0.0,0.018186,0.0181856433333


Make all of the energies numeric

In [3]:
for col in oqmd_data.columns:
    if col == 'comp': continue
    oqmd_data[col] = pd.to_numeric(oqmd_data[col], errors='coerce')

Eliminate entries with weird formation enthalpies

In [4]:
oqmd_data.query('delta_e > -20 and delta_e < 5', inplace=True)

Generate the composition object of each entry

In [5]:
oqmd_data['comp_obj'] = oqmd_data['comp'].apply(lambda x: Composition(x))

In [6]:
oqmd_data['pretty_comp'] = oqmd_data['comp_obj'].apply(lambda x: x.reduced_formula)

C:\Users\ward6\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:409: UserWarning: No electronegativity for Ne. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
C:\Users\ward6\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:409: UserWarning: No electronegativity for He. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
C:\Users\ward6\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:409: UserWarning: No electronegativity for Ar. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)


Get only the lowest-energy entry at each composition

In [7]:
oqmd_data.sort_values('delta_e', ascending=True, inplace=True)
oqmd_data.drop_duplicates('pretty_comp', keep='first', inplace=True)
print('Reduced dataset to %d entries'%len(oqmd_data))

Reduced dataset to 275701 entries


## Identify the systems with large numbers of entries
We want to find a system with a large amount of testing data

In [8]:
oqmd_data['nelems'] = oqmd_data['comp_obj'].apply(lambda x: len(x))

In [9]:
oqmd_data['system'] = oqmd_data['comp_obj'].apply(lambda x: "-".join([y.symbol for y in x]))

Get the top-10 most frequent systems

In [10]:
oqmd_data['system'].value_counts()[:10]

Mn-Na-O    20
O-Ti       18
O-V        18
Li-O-V     17
Fe-Na-O    17
H-O-V      17
C-H-N-O    16
Al-Mg      16
Na-O-V     16
Li-Mn-O    16
Name: system, dtype: int64

*Finding*: Mn-Na-O and Fe-Na-O are the most common ternaries. So, let's choose the Na-Fe-Mn-O quaternary as a hold-out. Also, there are a large number of Ti-O binary compounds, so let's use that one as the pairwise interaction holdout

## Exclude a Quaternary Diagram
Exclude the NaFeMnO data as the hold-out set

In [11]:
my_system = ["Na", "Fe", "Mn", "O"]

In [12]:
def get_test_data(elems):
    """Get the data that is in any of the phase diagrams that are subsets of a certain system
    
    Ex: For Na-Fe-O, these are Na-Fe-O, Na-Fe, Na-O, Fe-O, Na-Fe, Na, Fe, O
    
    :param elems: iterable of strs, phase diagram of interest
    :return: subset of OQMD in the constituent systems"""
    
    # Generate the constituent systems
    systems = set()
    for comb in product(*[elems,]*len(elems)):
        sys = "-".join(sorted(set(comb)))
        systems.add(sys)
    
    # Query for the data
    return oqmd_data.query(' or '.join('system=="%s"'%s for s in systems))

In [13]:
test_set = get_test_data(my_system)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Gathered a test set with 96 entries


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability,comp_obj,pretty_comp,nelems,system
361271,Fe1Mn1O6,-5.705954,10.7402,0.375689,0.000,-0.692279,0.482018,"(Fe, Mn, O)",MnFeO6,3,Fe-Mn-O
269091,Mn1,-9.026903,10.6840,0.096493,0.000,0.000000,0.000000,(Mn),Mn,1,Mn
346958,Na2O3,-4.088824,12.5533,NaN,0.000,-0.912622,0.285744,"(Na, O)",Na2O3,2,Na-O
434812,Fe1O3,-5.732222,10.4354,0.875159,0.000,-0.830062,0.204655,"(Fe, O)",FeO3,2,Fe-O
33714,Na1O1,-4.194173,12.0642,NaN,1.959,-1.336257,-0.081094,"(Na, O)",Na2O2,2,Na-O


Measure the standard deviation and MAD of test set (useful for evaluating model performance)

In [14]:
mad = np.abs(test_set['delta_e'] - test_set['delta_e'].mean()).mean()
std = test_set['delta_e'].std()
print('MAD: {:.3f} eV/atom'.format(mad))
print('Std Dev: {:.3f} eV/atom'.format(std))

MAD: 0.792 eV/atom
Std Dev: 0.965 eV/atom


Remove these entries from the dataset at large

In [15]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Training set size is 275605 entries


### Save the data in Magpie-friendly format
We will be using Magpie to generate features

In [16]:
def save_magpie(data, path):
    """Save a dataframe in a magpie-friendly format
    
    :param data: pd.DataFrame, data to be saved
    :param path: str, output path"""
    
    data[['comp','delta_e']].to_csv(path, index=False, sep=' ')

In [17]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%(''.join(my_system))))

In [18]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%(''.join(my_system))))

## Generate the hold-pairwise out dataset

In [19]:
my_pair = ['Ti', 'O']

In [20]:
def get_test_data(elems):
    """Get the data that contains all of a certain set of elements.
        
    :param elems: iterable of strs, elems to exclude
    :return: subset of OQMD in the constituent systems"""
    
    # Process the dataset
    hit = oqmd_data['system'].apply(lambda x: all([e in x.split("-") for e in elems]))
    return oqmd_data[hit]

In [21]:
test_set = get_test_data(my_pair)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Gathered a test set with 561 entries


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability,comp_obj,pretty_comp,nelems,system
111840,Mg1O4Ti2,-8.069274,10.81370,0.162796,0.000,-3.082515,-0.008433,"(Mg, O, Ti)",MgTi2O4,3,Mg-O-Ti
32013,O3Ti1Zn1,-7.095855,10.37400,-0.000065,2.955,-2.609167,-0.000974,"(O, Ti, Zn)",TiZnO3,3,O-Ti-Zn
335163,Li3Nb3O12Ti2,-7.930633,9.69378,0.100056,0.000,-2.669838,0.189520,"(Li, Nb, O, Ti)",Li3Ti2Nb3O12,4,Li-Nb-O-Ti
434912,O3Ti1,-7.426399,10.76590,0.125017,0.263,-2.113115,0.320940,"(O, Ti)",TiO3,2,O-Ti
17410,Nd2O11Ti4,-8.719405,11.87590,NaN,3.098,-3.442313,-0.025459,"(Nd, O, Ti)",Nd2Ti4O11,3,Nd-O-Ti


Measure the standard deviation and MAD of test set (useful for evaluating model performance)

In [22]:
mad = np.abs(test_set['delta_e'] - test_set['delta_e'].mean()).mean()
std = test_set['delta_e'].std()
print('MAD: {:.3f} eV/atom'.format(mad))
print('Std Dev: {:.3f} eV/atom'.format(std))

MAD: 0.478 eV/atom
Std Dev: 0.605 eV/atom


Remove these entries from the dataset at large

In [23]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Training set size is 275140 entries


Save the data

In [24]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%('-'.join(my_pair))))

In [25]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%('-'.join(my_pair))))